In [1]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
user_balance = pd.read_csv("data/user_balance_table.csv").reset_index(drop=True)

In [4]:
user_balance['date'] = pd.to_datetime(user_balance['report_date'], format="%Y%m%d")
user_balance['year'] = user_balance['date'].dt.year
user_balance['month'] = user_balance['date'].dt.month
user_balance['week'] = user_balance['date'].dt.week
user_balance['day'] = user_balance['date'].dt.day
user_balance['weekday'] = user_balance['date'].dt.weekday

In [5]:
total_balance = user_balance.groupby(['date'])['total_purchase_amt', 'total_redeem_amt'].sum()
total_balance.reset_index(inplace=True)
# total_balance[user_balance['date'] >= '2014-03-01'].reset_index(drop=True)

In [6]:
total_balance['date'] = pd.to_datetime(total_balance['date'], format="%Y%m%d")
total_balance['year'] = total_balance['date'].dt.year
total_balance['month'] = total_balance['date'].dt.month
total_balance['week'] = total_balance['date'].dt.week
total_balance['day'] = total_balance['date'].dt.day
total_balance['weekday'] = total_balance['date'].dt.weekday

In [7]:
total_balance

,date,total_purchase_amt,total_redeem_amt,year,month,week,day,weekday
0,2013-07-01,32488348,5525022,2013,7,27,1,0
1,2013-07-02,29037390,2554548,2013,7,27,2,1
2,2013-07-03,27270770,5953867,2013,7,27,3,2
3,2013-07-04,18321185,6410729,2013,7,27,4,3
4,2013-07-05,11648749,2763587,2013,7,27,5,4
...,...,...,...,...,...,...,...,...
422,2014-08-27,302194801,468164147,2014,8,35,27,2
423,2014-08-28,245082751,297893861,2014,8,35,28,3
424,2014-08-29,267554713,273756380,2014,8,35,29,4
425,2014-08-30,199708772,196374134,2014,8,35,30,5


In [8]:
user_information = pd.read_csv("data/user_profile_table.csv")

In [9]:
user_information

,user_id,sex,city,constellation
0,2,1,6411949,狮子座
1,12,1,6412149,摩羯座
2,22,1,6411949,双子座
3,23,1,6411949,双鱼座
4,25,1,6481949,双鱼座
...,...,...,...,...
28036,28017,0,6412149,摩羯座
28037,28018,0,6411949,摩羯座
28038,28022,1,6081949,双鱼座
28039,28034,1,6301949,天蝎座


In [10]:
def get_holiday_set():
    holiday_set = set()
    # 清明节
    holiday_set = holiday_set | {datetime.date(2014, 4, 5), datetime.date(2014, 4, 6), datetime.date(2014, 4, 7)}
    # 劳动节
    holiday_set = holiday_set | {datetime.date(2014, 5, 1), datetime.date(2014, 5, 2), datetime.date(2014, 5, 3)}
    # 端午节
    holiday_set = holiday_set | {datetime.date(2014, 5, 31), datetime.date(2014, 6, 1), datetime.date(2014, 6, 2)}
    # 中秋节
    holiday_set = holiday_set | {datetime.date(2014, 9, 6), datetime.date(2014, 9, 7), datetime.date(2014, 9, 8)}
    # 国庆节
    holiday_set = holiday_set | {datetime.date(2014, 10, 1), datetime.date(2014, 10, 2), datetime.date(2014, 10, 3), \
                                 datetime.date(2014, 10, 4), datetime.date(2014, 10, 5), datetime.date(2014, 10, 6), \
                                 datetime.date(2014, 10, 7)}
    # 中秋节
    holiday_set = holiday_set | {datetime.date(2013, 9, 19), datetime.date(2013, 9, 20), datetime.date(2013, 9, 21)}
    # 国庆节
    holiday_set = holiday_set | {datetime.date(2013, 10, 1), datetime.date(2013, 10, 2), datetime.date(2013, 10, 3), \
                                 datetime.date(2013, 10, 4), datetime.date(2013, 10, 5), datetime.date(2013, 10, 6), \
                                 datetime.date(2013, 10, 7)}
    return holiday_set

In [11]:
def extract_is_feature(data: pd.DataFrame) -> pd.DataFrame:
    total_balance = data.copy().reset_index(drop=True)

    # 是否是Weekend
    total_balance['is_weekend'] = 0
    total_balance.loc[total_balance['weekday'].isin((5, 6)), 'is_weekend'] = 1
    # 是否是假期
    total_balance['is_holiday'] = 0
    total_balance.loc[total_balance['date'].isin(get_holiday_set()), 'is_holiday'] = 1

    # 是否是节假日的第一天
#     last_day_flag = 0
#     total_balance['is_firstday_of_holiday'] = 0
#     for index, row in total_balance.iterrows():
#         if last_day_flag == 0 and row['is_holiday'] == 1:
#             total_balance.loc[index, 'is_firstday_of_holiday'] = 1
#         last_day_flag = row['is_holiday']

#     是否是节假日的最后一天
    total_balance['is_lastday_of_holiday'] = 0
    for index, row in total_balance.iterrows():
        if row['is_holiday'] == 1 and total_balance.loc[index + 1, 'is_holiday'] == 0:
            total_balance.loc[index, 'is_lastday_of_holiday'] = 1

    # 是否是节假日后的上班第一天
#     total_balance['is_firstday_of_work'] = 0
#     last_day_flag = 0
#     for index, row in total_balance.iterrows():
#         if last_day_flag == 1 and row['is_holiday'] == 0:
#             total_balance.loc[index, 'is_firstday_of_work'] = 1
#         last_day_flag = row['is_lastday_of_holiday']

    # 是否不用上班
    total_balance['is_work'] = 1
    total_balance.loc[(total_balance['is_holiday'] == 1) | (total_balance['is_weekend'] == 1), 'is_work'] = 0
    special_work_day_set = {datetime.date(2014, 5, 4), datetime.date(2014, 9, 28)}
    total_balance.loc[total_balance['date'].isin(special_work_day_set), 'is_work'] = 1

    # 是否明天要上班
    total_balance['is_gonna_work_tomorrow'] = 0
    for index, row in total_balance.iterrows():
        if index == len(total_balance) - 1:
            break
        if row['is_work'] == 0 and total_balance.loc[index + 1, 'is_work'] == 1:
            total_balance.loc[index, 'is_gonna_work_tomorrow'] = 1

    # 昨天上班了吗
#     total_balance['is_worked_yestday'] = 0
#     for index, row in total_balance.iterrows():
#         if index <= 1:
#             continue
#         if total_balance.loc[index - 1, 'is_work'] == 1:
#             total_balance.loc[index, 'is_worked_yestday'] = 1

    # 是否是放假前一天
#     total_balance['is_lastday_of_workday'] = 0
#     for index, row in total_balance.iterrows():
#         if index == len(total_balance) - 1:
#             break
#         if row['is_holiday'] == 0 and total_balance.loc[index + 1, 'is_holiday'] == 1:
#             total_balance.loc[index, 'is_lastday_of_workday'] = 1

    # 是否是月初第一天
#     total_balance['is_firstday_of_month'] = 0
#     total_balance.loc[total_balance['day'] == 1, 'is_firstday_of_month'] = 1

    # 是否是月初第二天
#     total_balance['is_secondday_of_month'] = 0
#     total_balance.loc[total_balance['day'] == 2, 'is_secondday_of_month'] = 1

    # 是否是月初
#     total_balance['is_headmonth'] = 0
#     total_balance.loc[total_balance['day'] <= 10, 'is_headmonth'] = 1

    # 是否是月中
#     total_balance['is_midmonth'] = 0
#     total_balance.loc[(10 < total_balance['day']) & (total_balance['day'] <= 20), 'is_midmonth'] = 1

    # 是否是月末
#     total_balance['is_tailmonth'] = 0
#     total_balance.loc[20 < total_balance['day'], 'is_tailmonth'] = 1

    # 是否是每个月第一周
#     total_balance['is_first_week'] = 0
#     total_balance.loc[total_balance['week'] % 4 == 1, 'is_first_week'] = 1

    # 是否是每个月第二周
#     total_balance['is_second_week'] = 0
#     total_balance.loc[total_balance['week'] % 4 == 2, 'is_second_week'] = 1

    # 是否是每个月第三周
#     total_balance['is_third_week'] = 0
#     total_balance.loc[total_balance['week'] % 4 == 3, 'is_third_week'] = 1

    # 是否是每个月第四周
#     total_balance['is_fourth_week'] = 0
#     total_balance.loc[total_balance['week'] % 4 == 0, 'is_fourth_week'] = 1

    return total_balance.reset_index(drop=True)

In [12]:
total_balance = extract_is_feature(total_balance)

In [13]:
def encode_data(data: pd.DataFrame, feature_name: str = 'weekday', encoder=OneHotEncoder()):
    total_balance = data.copy()
    week_feature = encoder.fit_transform(np.array(total_balance[feature_name]).reshape(-1, 1)).toarray()
    week_feature = pd.DataFrame(week_feature,
                                columns=[feature_name + '_onehot_' + str(x) for x in range(len(week_feature[0]))])
    # featureWeekday = pd.concat([total_balance, week_feature], axis = 1).drop(feature_name, axis=1)
    featureWeekday = pd.concat([total_balance, week_feature], axis=1)
    return featureWeekday

total_balance = encode_data(total_balance)

In [14]:
total_balance

,date,total_purchase_amt,total_redeem_amt,year,month,week,day,weekday,is_weekend,is_holiday,is_lastday_of_holiday,is_work,is_gonna_work_tomorrow,weekday_onehot_0,weekday_onehot_1,weekday_onehot_2,weekday_onehot_3,weekday_onehot_4,weekday_onehot_5,weekday_onehot_6
0,2013-07-01,32488348,5525022,2013,7,27,1,0,0,0,0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013-07-02,29037390,2554548,2013,7,27,2,1,0,0,0,1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2013-07-03,27270770,5953867,2013,7,27,3,2,0,0,0,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2013-07-04,18321185,6410729,2013,7,27,4,3,0,0,0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2013-07-05,11648749,2763587,2013,7,27,5,4,0,0,0,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,2014-08-27,302194801,468164147,2014,8,35,27,2,0,0,0,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
423,2014-08-28,245082751,297893861,2014,8,35,28,3,0,0,0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
424,2014-08-29,267554713,273756380,2014,8,35,29,4,0,0,0,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
425,2014-08-30,199708772,196374134,2014,8,35,30,5,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
features = total_balance[[x for x in total_balance.columns if x not in ['week', 'year', 'month', 'weekday', 'day']]]

In [16]:
temp = np.abs(features[[x for x in features.columns
                       if x not in ['total_redeem_amt', 'date']]].corr('spearman')['total_purchase_amt'])
feature_low_correlation = list(set(temp[temp < 0.1].index))


In [17]:
feature_low_correlation

['weekday_onehot_3',
 'is_lastday_of_holiday',
 'weekday_onehot_4',
 'weekday_onehot_2']

In [18]:
def extract_distance_feature(data: pd.DataFrame):
    total_balance = data.copy()

    # 距离放假还有多少天
    total_balance['dis_to_nowork'] = 0
    for index, row in total_balance.iterrows():
        if row['is_work'] == 0:
            step = 1
            flag = 1
            while flag:
                if index - step >= 0 and total_balance.loc[index - step, 'is_work'] == 1:
                    total_balance.loc[index - step, 'dis_to_nowork'] = step
                    step += 1
                else:
                    flag = 0

    total_balance['dis_from_nowork'] = 0
    step = 0
    for index, row in total_balance.iterrows():
        step += 1
        if row['is_work'] == 1:
            total_balance.loc[index, 'dis_from_nowork'] = step
        else:
            step = 0

    # 距离上班还有多少天
    total_balance['dis_to_work'] = 0
    for index, row in total_balance.iterrows():
        if row['is_work'] == 1:
            step = 1
            flag = 1
            while flag:
                if index - step >= 0 and total_balance.loc[index - step, 'is_work'] == 0:
                    total_balance.loc[index - step, 'dis_to_work'] = step
                    step += 1
                else:
                    flag = 0

    total_balance['dis_from_work'] = 0
    step = 0
    for index, row in total_balance.iterrows():
        step += 1
        if row['is_work'] == 0:
            total_balance.loc[index, 'dis_from_work'] = step
        else:
            step = 0

    # 距离节假日还有多少天
#     total_balance['dis_to_holiday'] = 0
#     for index, row in total_balance.iterrows():
#         if row['is_holiday'] == 1:
#             step = 1
#             flag = 1
#             while flag:
#                 if index - step >= 0 and total_balance.loc[index - step, 'is_holiday'] == 0:
#                     total_balance.loc[index - step, 'dis_to_holiday'] = step
#                     step += 1
#                 else:
#                     flag = 0

    total_balance['dis_from_holiday'] = 0
    step = 0
    for index, row in total_balance.iterrows():
        step += 1
        if row['is_holiday'] == 0:
            total_balance.loc[index, 'dis_from_holiday'] = step
        else:
            step = 0

    # 距离节假日最后一天还有多少天
    total_balance['dis_to_holiendday'] = 0
    for index, row in total_balance.iterrows():
        if row['is_lastday_of_holiday'] == 1:
            step = 1
            flag = 1
            while flag:
                if index - step >= 0 and total_balance.loc[index - step, 'is_lastday_of_holiday'] == 0:
                    total_balance.loc[index - step, 'dis_to_holiendday'] = step
                    step += 1
                else:
                    flag = 0

    total_balance['dis_from_holiendday'] = 0
    step = 0
    for index, row in total_balance.iterrows():
        step += 1
        if row['is_lastday_of_holiday'] == 0:
            total_balance.loc[index, 'dis_from_holiendday'] = step
        else:
            step = 0

    # 距离月初第几天
#     total_balance['dis_from_startofmonth'] = np.abs(total_balance['day'])

    # 距离月的中心点有几天
#     total_balance['dis_from_middleofmonth'] = np.abs(total_balance['day'] - 15)

    # 距离星期的中心有几天
#     total_balance['dis_from_middleofweek'] = np.abs(total_balance['weekday'] - 3)

    # 距离星期日有几天
    total_balance['dis_from_endofweek'] = np.abs(total_balance['weekday'] - 6)

    return total_balance

In [19]:
total_balance = extract_distance_feature(total_balance)

In [20]:
feature = total_balance[[x for x in total_balance.columns if x not in ['week', 'year', 'month', 'weekday', 'day']]]

In [21]:
temp = np.abs(feature[[x for x in feature.columns
                       if ('dis' in x) | (x in ['total_purchase_amt'])]].corr()['total_purchase_amt'])
feature_low_correlation += list(set(temp[temp < 0.1].index))

In [22]:
feature_low_correlation

['weekday_onehot_3',
 'is_lastday_of_holiday',
 'weekday_onehot_4',
 'weekday_onehot_2']

In [23]:
def extract_peak_feature(data: pd.DataFrame):
    total_balance = data.copy()
    # 距离purchase波峰（即周二）有几天
    total_balance['dis_from_purchase_peak'] = np.abs(total_balance['weekday'] - 1)

    # 距离purchase波谷（即周日）有几天，与dis_from_endofweek相同
    total_balance['dis_from_purchase_valley'] = np.abs(total_balance['weekday'] - 6)

    return total_balance

In [24]:
total_balance = extract_peak_feature(total_balance)

In [25]:
total_balance.columns

Index(['date', 'total_purchase_amt', 'total_redeem_amt', 'year', 'month',
       'week', 'day', 'weekday', 'is_weekend', 'is_holiday',
       'is_lastday_of_holiday', 'is_work', 'is_gonna_work_tomorrow',
       'weekday_onehot_0', 'weekday_onehot_1', 'weekday_onehot_2',
       'weekday_onehot_3', 'weekday_onehot_4', 'weekday_onehot_5',
       'weekday_onehot_6', 'dis_to_nowork', 'dis_from_nowork', 'dis_to_work',
       'dis_from_work', 'dis_from_holiday', 'dis_to_holiendday',
       'dis_from_holiendday', 'dis_from_endofweek', 'dis_from_purchase_peak',
       'dis_from_purchase_valley'],
      dtype='object')